In [1]:
import numpy as np
import xarray as xr

datadir = '/gpfs/work/nonnenma/data/forecast_predictability/weatherbench/5_625deg/'
res_dir = '/gpfs/work/nonnenma/results/forecast_predictability/weatherbench/5_625deg/'

var_dict = {'geopotential': ('z', [1,10,100,200,300,400,500,600,700,850,1000]),
           'temperature': ('t', [1,10,100,200,300,400,500,600,700,850,1000]),
           'u_component_of_wind': ('u', [1,10,100,200,300,400,500,600,700,850,1000]), 
           'v_component_of_wind': ('v', [1,10,100,200,300,400,500,600,700,850,1000]),
           'specific_humidity': ('q', [1,10,100,200,300,400,500,600,700,850,1000]),
           'relative_humidity': ('r', [1,10,100,200,300,400,500,600,700,850,1000]),
           'vorticity': ('vo', [1,10,100,200,300,400,500,600,700,850,1000]),
           'potential_vorticity': ('pv', [1,10,100,200,300,400,500,600,700,850,1000]),
           'total_cloud_cover': ('tcc', [None]),
           'total_precipitation': ('tp', [None]),
           'toa_incident_solar_radiation': ('tisr', [None]),
           'rel_time': ('time_map', [None]),
           'constants': ['lsm','orography','lat2d']
           }

In [2]:
x = xr.merge(
[xr.open_mfdataset(f'{datadir}/{var}/*.nc', combine='by_coords')
 for var in var_dict.keys()],
fill_value=0  # For the 'tisr' NaNs
)
x

<xarray.Dataset>
Dimensions:    (lat: 32, level: 11, lon: 64, time: 350640)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
  * lat        (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * lon        (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * level      (level) int32 1 10 100 200 300 400 500 600 700 850 1000
Data variables:
    z          (time, level, lat, lon) float32 dask.array<chunksize=(8760, 11, 32, 64), meta=np.ndarray>
    t          (time, level, lat, lon) float32 dask.array<chunksize=(8760, 11, 32, 64), meta=np.ndarray>
    u          (time, level, lat, lon) float32 dask.array<chunksize=(8760, 11, 32, 64), meta=np.ndarray>
    v          (time, level, lat, lon) float32 dask.array<chunksize=(8760, 11, 32, 64), meta=np.ndarray>
    q          (time, level, lat, lon) float32 dask.array<chunksize=(8760, 11, 32, 64), meta=np.ndarray>
    r          (time, level, lat, lon) float32 dask.array<chunksize=(8760, 11, 32, 64), meta=np.ndarray>
    vo         (time, level, lat, lon) float32 dask.array<chunksize=(8760, 11, 32, 64), meta=np.ndarray>
    pv         (time, level, lat, lon) float32 dask.array<chunksize=(8760, 11, 32, 64), meta=np.ndarray>
    tcc        (time, lat, lon) float32 dask.array<chunksize=(8760, 32, 64), meta=np.ndarray>
    tp         (time, lat, lon) float32 dask.array<chunksize=(8760, 32, 64), meta=np.ndarray>
    tisr       (time, lat, lon) float32 dask.array<chunksize=(8760, 32, 64), meta=np.ndarray>
    time_map   (time, lat, lon) float32 dask.array<chunksize=(350640, 32, 64), meta=np.ndarray>
    orography  (lat, lon) float32 dask.array<chunksize=(32, 64), meta=np.ndarray>
    lsm        (lat, lon) float32 dask.array<chunksize=(32, 64), meta=np.ndarray>
    slt        (lat, lon) float32 dask.array<chunksize=(32, 64), meta=np.ndarray>
    lat2d      (lat, lon) float64 dask.array<chunksize=(32, 64), meta=np.ndarray>
    lon2d      (lat, lon) float64 dask.array<chunksize=(32, 64), meta=np.ndarray>

In [ ]:
"""
# create and store maps of relative time in hours from 1979-01-01T00:00:00.00
all_time = np.asarray(np.arange(x.time.size).reshape(-1,1,1) * np.ones((x.time.size,32,64)), dtype=np.float32)
xtime = xr.DataArray(
        all_time, 
        name='time_map',
        dims=['time', 'lat', 'lon'], 
        coords={'time': x.time, 'lon': x.lon, 'lat': x.lat}) 
xtime.to_netcdf(f'{datadir}/rel_time/rel_time_5.625deg.nc')

# compute and store means and stds for later z-scoring
rel_time_mean = xtime.mean(('time', 'lat', 'lon')).compute()
rel_time_std = xtime.std(('time', 'lat', 'lon')).compute()
generic_level = xr.DataArray([1], coords={'level': [1]}, dims=['level'])
rel_time_mean = rel_time_mean.expand_dims({'level': generic_level}, 0)
rel_time_std = rel_time_std.expand_dims({'level': generic_level}, 0)

train_years = ('1979', '2015')
var = 'rel_time'
rel_time_mean.to_netcdf(f'{res_dir}/{var}/mean_{train_years[0]}_{train_years[1]}.nc')
rel_time_std.to_netcdf(f'{res_dir}/{var}/std_{train_years[0]}_{train_years[1]}.nc')
xtime
"""

In [3]:
ds, dtype = x, np.float32
data = []
level_names = []
generic_level = xr.DataArray([1], coords={'level': [1]}, dims=['level'])
for long_var, params in var_dict.items():
    if long_var == 'constants':
        for var in params:
            data.append(ds[var].expand_dims(
                {'level': generic_level, 'time': ds.time}, (1, 0)
            ).astype(dtype))
            level_names.append(var)
    else:
        var, levels = params
        try:
            data.append(ds[var].sel(level=levels))
            level_names += [f'{var}_{level}' for level in levels]
        except ValueError:
            data.append(ds[var].expand_dims({'level': generic_level}, 1))
            level_names.append(var)
data = xr.concat(data, 'level')  # .transpose('time', 'lat', 'lon', 'level')
data['level_names'] = xr.DataArray(level_names, dims=['level'], coords={'level': data.level})        

In [4]:
data

<xarray.DataArray 'z' (time: 350640, level: 95, lat: 32, lon: 64)>
dask.array<concatenate, shape=(350640, 95, 32, 64), dtype=float32, chunksize=(8784, 11, 32, 64), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
  * lat          (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * lon          (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * level        (level) int64 1 10 100 200 300 400 500 600 ... 1 1 1 1 1 1 1
    level_names  (level) <U9 'z_1' 'z_10' 'z_100' ... 'lsm' 'orography' 'lat2d'
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [5]:
allvalues = data.values
allvalues.shape

(350640, 95, 32, 64)

In [6]:
np.save(datadir + '5_625deg_all', allvalues, allow_pickle=False)

In [7]:
np.save(datadir + '5_625deg_all_level_names', data['level_names'].values)

In [8]:
from src.pytorch.Dataset import load_mean_std
train_years = ('1979', '2015')

mean, std, level, level_names = load_mean_std(res_dir, var_dict, train_years)
for i in np.where([ln[-5:]=='_None' for ln in level_names])[0]: # some cleanup
    level_names[i] = level_names[i][:-5] # discrepancy in code between code for single-level levels: '1' vs 'None'
assert np.all( np.array(level_names) == data['level_names'])
mean.shape, std.shape, len(level_names)

((95,), (95,), 95)

In [9]:
allvalues -= mean.reshape(1,-1,1,1) # in-place feels dangerous, 
allvalues /= std.reshape(1,-1,1,1)  # but allvalues hardly fits into memory twice

In [10]:
np.save(datadir + '5_625deg_all_zscored', allvalues, allow_pickle=False)